In [1]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, String, Float, DateTime
import psycopg2

In [2]:
# Caminho para o arquivo de chave da conta de serviço - Lucas disponibilizou no Whatsapp do grupo
service_account_path = 'tfm-sa.json'

# Criar objeto de credenciais
credentials = service_account.Credentials.from_service_account_file(service_account_path)

In [3]:
project_id = 'perseverance-332400'
dataset_id = 'TFM'
table_id = 'ds_market'
full_table_id = f'{project_id}.{dataset_id}.{table_id}'
client = bigquery.Client(project='perseverance-332400', credentials=credentials)

Total de vendas para cada loja, agrupadas por região e departamento, e calcule a contribuição percentual de cada loja para o total de vendas dentro do grupo.

In [ ]:
SELECT
  region,
  department,
  store,
  SUM(sales) AS total_sales,
  (SUM(sales) / SUM(SUM(sales)) OVER (PARTITION BY region, department)) * 100 AS sales_contribution_percentage
FROM
  `TFM.ds_market`
GROUP BY
  region,
  department,
  store;

 Total de vendas para cada combinação de category_x e category_y, agrupadas por região e semana do ano, e calcule a contribuição percentual de cada combinação para o total de vendas dentro do grupo.

In [ ]:
SELECT
  region,
  yearweek,
  category_x,
  category_y,
  SUM(sales) AS total_sales,
  (SUM(sales) / SUM(SUM(sales)) OVER (PARTITION BY region, yearweek)) * 100 AS sales_contribution_percentage
FROM
  `TFM.ds_market`
GROUP BY
  region,
  yearweek,
  category_x,
  category_y;

Os 5 itens com as maiores vendas para uma determinada região e semana do ano, e calcule a contribuição percentual de cada item para o total de vendas dentro do grupo.

In [ ]:
SELECT
  region,
  yearweek,
  item,
  SUM(sales) AS total_sales,
  (SUM(sales) / SUM(SUM(sales)) OVER (PARTITION BY region, yearweek)) * 100 AS sales_contribution_percentage
FROM
  `TFM.ds_market`
WHERE
  region = 'US'
  AND yearweek = '2023-W01'
GROUP BY
  region,
  yearweek,
  item
ORDER BY
  total_sales DESC
LIMIT
  5;

O total de vendas para cada loja, agrupadas por região e semana do ano, e calcule a variação percentual ano a ano nas vendas para cada loja.

In [ ]:
SELECT
  region,
  yearweek,
  store,
  SUM(sales) AS total_sales,
  (SUM(sales) / SUM(SUM(sales)) OVER (PARTITION BY region)) * 100 AS sales_contribution_percentage
FROM
  `TFM.ds_market`
GROUP BY
  region,
  yearweek,
  store;

As 5 lojas com as maiores vendas médias para uma determinada região e semana do ano.

In [ ]:
SELECT
  region,
  yearweek,
  store,
  AVG(sales) AS avg_sales
FROM
  `TFM.ds_market`
WHERE
  region = 'US'
  AND yearweek = '2023-W01'
GROUP BY
  region,
  yearweek,
  store
ORDER BY
  avg_sales DESC
LIMIT
  5;

Encontre a correlação entre vendas e preço de venda para cada região.

In [ ]:
SELECT
  region,
  CORR(sales, sell_price) AS sales_sell_price_corr
FROM
  `TFM.ds_market`
GROUP BY
  region;

Encontre os 5 itens com as maiores vendas para uma determinada região e semana do ano.

In [ ]:
SELECT
  item,
  SUM(sales) AS total_sales
FROM
  `TFM.ds_market`
WHERE
  region = 'US'
  AND yearweek = '2023-W01'
GROUP BY
  item
ORDER BY
  total_sales DESC
LIMIT
  5;

Encontre o total de vendas para cada loja, agrupadas por região e departamento.

In [ ]:
SELECT
  region,
  department,
  SUM(sales) AS total_sales
FROM
  `TFM.ds_market`
GROUP BY
  region,
  department;

Encontre o preço de venda médio para cada category_x, agrupado por região e semana do ano.

In [ ]:
SELECT
  region,
  category_y,
  store,
  MAX(sales) AS max_sales
FROM
  `TFM.ds_market`
GROUP BY
  region,
  category_y,
  store;

Encontre o desvio padrão das vendas para cada loja, agrupadas por região e ano-semana.

In [ ]:
SELECT
  region,
  yearweek,
  store,
  STDDEV(sales) AS stddev_sales
FROM
  `TFM.ds_market`
GROUP BY
  region,
  yearweek,
  store;

Calcule a média de vendas por região para cada ano-semana

In [ ]:
SELECT
  region,
  yearweek,
  AVG(sales) AS average_sales
FROM
  `TFM.ds_market`
WHERE
  date BETWEEN '2022-01-01'
  AND '2022-12-31'
GROUP BY
  region,
  yearweek;

Encontre as vendas máximas para cada combinação de loja e categoria.

In [ ]:
SELECT
  department,
  STDDEV_SAMP(sales) AS sales_standard_deviation
FROM
  `TFM.ds_market`
WHERE
  date BETWEEN '2022-01-01'
  AND '2022-12-31'
GROUP BY
  department;

Encontre os 5 itens com o maior preço médio de venda

In [ ]:
SELECT
  item,
  AVG(sell_price) AS average_sell_price
FROM
  `TFM.ds_market`
WHERE
  date BETWEEN '2022-01-01'
  AND '2022-12-31'
GROUP BY
  item
ORDER BY
  average_sell_price DESC
LIMIT
  5;

 A correlação entre vendas e preço de venda

In [ ]:
SELECT
  CORR(sales, sell_price) AS sales_sell_price_correlation
FROM
  `TFM.ds_market`
WHERE
  date BETWEEN '2022-01-01'
  AND '2022-12-31';

Calcule a média de vendas por loja para cada combinação de região e ano-semana

In [ ]:
SELECT
  region,
  yearweek,
  store,
  AVG(sales) AS average_sales
FROM
  `TFM.ds_market`
WHERE
  date BETWEEN '2022-01-01'
  AND '2022-12-31'
GROUP BY
  region,
  yearweek,
  store;

 o preço médio de venda para cada combinação de categoria_x e código da loja.

In [ ]:
SELECT
  category_x,
  store_code,
  AVG(sell_price) AS avg_sell_price
FROM
  `TFM.ds_market`
GROUP BY
  category_x,
  store_code;

Calcule o total de vendas para cada combinação de ano-semana e categoria_x, e classifique-os dentro de cada ano-semana

In [ ]:
SELECT
  region,
  yearweek,
  sales,
  (sales - LAG(sales, 1, 0) OVER (PARTITION BY region ORDER BY yearweek)) * 100.0 / LAG(sales, 1, 0) OVER (PARTITION BY region ORDER BY yearweek) AS sales_change_percent
FROM
  `TFM.ds_market`
WHERE
  yearweek > '20230101';

As vendas acumuladas para cada combinação de código da loja e categoria_y ao longo do tempo.

In [ ]:
SELECT
  store_code,
  category_y,
  date,
  SUM(sales) OVER (PARTITION BY store_code, category_y ORDER BY date) AS cumulative_sales
FROM
  `TFM.ds_market`
ORDER BY
  store_code,
  category_y,
  date;

Calcule a média móvel de 7 dias das vendas para cada região.

In [ ]:
SELECT
  region,
  date,
  AVG(sales) OVER (PARTITION BY region ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS moving_avg_sales
FROM
  `TFM.ds_market`
ORDER BY
  region,
  date;

O total de vendas para cada combinação de evento e departamento, e classifique-os dentro de cada evento

In [ ]:
SELECT
  event,
  department,
  SUM(sales) AS total_sales,
  RANK() OVER (PARTITION BY event ORDER BY SUM(sales) DESC) AS sales_rank
FROM
  `TFM.ds_market`
GROUP BY
  event,
  department
ORDER BY
  event,
  sales_rank;

Encontre as 3 regiões com o maior total de vendas para uma categoria específica.

In [ ]:
SELECT
  region,
  SUM(sales) AS total_sales
FROM
  `TFM.ds_market`
WHERE
  category_x = 'Category X'
GROUP BY
  region
ORDER BY
  total_sales DESC
LIMIT
  3;

Encontre as 5 lojas com o maior preço médio de venda para um departamento específico

In [ ]:
SELECT
  store_code,
  AVG(sell_price) AS avg_sell_price
FROM
  `TFM.ds_market`
WHERE
  department = 'Department X'
GROUP BY
  store_code
ORDER BY
  avg_sell_price DESC
LIMIT
  5;

Encontre os 3 eventos com o maior total de vendas para um item específico

In [ ]:
SELECT
  event,
  SUM(sales) AS total_sales
FROM
  `TFM.ds_market`
WHERE
  item = 'Item X'
GROUP BY
  event
ORDER BY
  total_sales DESC
LIMIT
  3;

Encontre as 5 categorias com a maior média de vendas para uma loja específica

In [ ]:
SELECT
  category_x,
  AVG(sales) AS avg_sales
FROM
  `TFM.ds_market`
WHERE
  store_code = 'Store X'
GROUP BY
  category_x
ORDER BY
  avg_sales DESC
LIMIT
  5;